In [51]:
from glob import glob
import simplejson as json
import requests
import jinja2
from xml.sax.saxutils import escape

## How to get the json files?
* Open the map on camping.info
* open the Chrome web inspector "Network Tab"
* zoom into the area of interest until campsites appear on the map
* look for the *mapchange* request in the web inspector. This is the json file containing the coordinates. 
* retrieve the json files for all areas of interest (or write a script that does it) and continue with this crawler. 

## get coordinates and campsite ids

In [19]:
campsites = {}
for f in glob("./mapchange_json/*.json"):
    with open(f) as fh:
        data = json.load(fh)
        for campsite in data['campsitesInside']:
            campsites[campsite['id']] = campsite

In [20]:
campsites

{3: {'id': 3, 'lat': 42.79136, 'lng': 17.8842, 'p': 0},
 6: {'id': 6, 'lat': 42.62449, 'lng': 18.20905, 'p': 0},
 7: {'id': 7, 'lat': 42.62416, 'lng': 18.18888, 'p': 0},
 17: {'id': 17, 'lat': 42.66203, 'lng': 18.07034, 'p': 1},
 18: {'id': 18, 'lat': 42.71519, 'lng': 17.97579, 'p': 0},
 145: {'id': 145, 'lat': 45.046, 'lng': 18.99686, 'p': 0},
 146: {'id': 146, 'lat': 48.949, 'lng': 15.721, 'p': 0},
 147: {'id': 147, 'lat': 48.90003, 'lng': 16.64998, 'p': 0},
 148: {'id': 148, 'lat': 48.90004, 'lng': 16.57098, 'p': 0},
 150: {'id': 150, 'lat': 48.91357, 'lng': 15.815, 'p': 0},
 152: {'id': 152, 'lat': 48.90951, 'lng': 17.3121, 'p': 0},
 207: {'id': 207, 'lat': 48.92083, 'lng': 16.02583, 'p': 0},
 209: {'id': 209, 'lat': 48.90805, 'lng': 16.83105, 'p': 0},
 273: {'id': 273, 'lat': 48.76724, 'lng': 17.18592, 'p': 0},
 772: {'id': 772, 'lat': 46.26053, 'lng': 10.4545, 'p': 0},
 781: {'id': 781, 'lat': 46.23861, 'lng': 10.46944, 'p': 0},
 782: {'id': 782, 'lat': 46.25528, 'lng': 9.56639, 

## get campsite details

In [21]:
url = "https://www.camping.info/ajax/getgo?goId={}"

In [22]:
for i, desc in campsites.items():
    r = requests.get(url.format(i))
    try: 
        details = r.json()
        desc.update(details)
    except json.JSONDecodeError:
        pass    

In [26]:
with open('results/campsites.json', 'w') as fh: 
    json.dump(campsites, fh)

In [28]:
with open('results/campsites.json', 'r') as fh:
    campsites = json.load(fh)

## convert to gpx and store

In [40]:
jinja_env = jinja2.Environment(loader=jinja2.FileSystemLoader('./templates/'))
gpx_template = jinja_env.get_template('gpx.xml.tpl')
wp_template = jinja_env.get_template('waypoint.xml.tpl')

In [54]:
def desc_from_dict(obj):
    return "\n".join([escape("{}: {}".format(k, v)) for k, v in obj.items()])    

In [62]:
waypoints = []
for id, campsite in campsites.items():
    wp_dict = {
        "lat": campsite['lat'],
        "lon": campsite['lng'],
        "name": escape(campsite['name']),
        "desc": desc_from_dict(campsite)
    }
    assert wp_dict["lat"] is not None
    assert wp_dict["lon"] is not None
    assert wp_dict["name"] != ""
    waypoints.append(wp_template.render(**wp_dict))

In [63]:
gpx_out = gpx_template.render(waypoints="\n".join(waypoints))

In [64]:
with open("results/campsites.gpx", 'w') as fh:
    fh.writelines(gpx_out)